# Score Combination

In [19]:
import os
import re
import pickle
import time
import inspect
import numpy as np
from IndexerCACM import *
from RelevantParser import *
from Query import *
from IRmodel import *
from Vector import *
from IRList import *
from EvalMeasure import *
from EvalIRModel import *
from Weighter import *
from copy import *
from Vector import *

### Get Indexers

In [20]:
# Processed collections 
collectionPath = 'data/cacm/cacm.txt'
collectionPath2 = 'data/cisi/cisi.txt'
queriesPath = 'data/cacm/cacm.qry'
relevantPath = 'data/cacm/cacm.rel'

In [21]:
indexer = IndexerCACM(collectionPath, ParserCACM())

# If Index and Inv Index aren't already builded
#indexer.createRepIndex()
#indexer.createRepInvIndex()
#indexer.createRepInvFromAll()
#indexer.createLinkIndex()

queriesIndexer = IndexerCACM(queriesPath, ParserCACM())

# If Index isn't already builded
#queriesIndexer.createRepIndex()

relevantIndexer = Indexer(relevantPath, RelevantParser())

# If Index and Inv Index aren't already builded
#relevantIndexer.createIndex()

In [22]:
q = query(1, queriesIndexer, relevantIndexer)
q2 = query(2, queriesIndexer, relevantIndexer)
q3 = query(3, queriesIndexer, relevantIndexer)
q4 = query(4, queriesIndexer, relevantIndexer)
print(q, q2, q3, q4)

(<Query.Query object at 0x7efc5068afd0>, <Query.Query object at 0x7efc247916d0>, <Query.Query object at 0x7efc24fc5210>, <Query.Query object at 0x7efc24791650>)


In [62]:
class Featurer(object):

    def __init__(self, indexer, path, irModel):
        
        self.indexer = indexer
        self.irModel = irModel
        self.path = path
        self.queryBool = len(inspect.getargspec(self.irModel.getScores).args)==2
        
        # load features
        if os.path.isfile(self.path):
            f = open(self.path)
            self.features = pickle.load(f)
            f.close()
        else:
            self.features = {}
        
    def getFeatures(self, id, query=None): 
        
            if self.queryBool:
                if query!=None:
                    if query.id not in self.features:
                        dic = {}
                        scores = self.irModel.getScores(query.el)
                        for doc in scores:
                            dic[doc['id']] = doc['score']

                        self.features[query.id] = dic
                        f = open(self.path, "w")
                        pickle.dump(self.features, f)
                        f.close()
                        
                    return self.features[query.id][str(id)]
            
            else:
                if self.features=={}:
                        scores = self.irModel.getScores()
                        for doc in scores:
                            dic[doc['id']] = doc['score']

                        self.features = dic
                        f = open(self.path, "w")
                        pickle.dump(self.features, f)
                        f.close()
                return self.features[id] 
        

In [63]:
class DocumentSize(object):
    
    def __init__(self, indexer):
        
        self.indexer = indexer
    
    def getScores(self):
        
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        
        i = 0
        for id in self.indexer.index:
            
            scores[i]['id'] = id
            scores[i]['score'] = self.getEfFromDoc(id)[-1]
        
        return scores
        

In [ ]:
class DocumentVocabularySize(object):
    
    def __init__(self, indexer):
        
        self.indexer = indexer
    
    def getScores(self):
        
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        
        i = 0
        for id in self.indexer.index:
            
            scores[i]['id'] = id
            scores[i]['score'] = len(self.getEfFromDoc(id)-1)
        
        return scores
        

In [64]:
class DocumentVocabularySize(object):
    
    def __init__(self, indexer):
        
        self.indexer = indexer
    
    def idf(self, elements):
        
        score = 0
        
        for element in elements:
            if element in self.indexer.invIndex:
                score += \
                self.nDoc / float(len(self.indexer.getDfFromEl(element))-1)
        
        return score
    
    def getScores(self):
        
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        
        i = 0
        for id in self.indexer.index:
            
            scores[i]['id'] = id
            scores[i]['score'] = idf(self.indexer.getEfFromDoc(id))
        
        return scores

In [56]:
class FeaturerList(object):
    

array([('1844', 22.0), ('1938', 17.0), ('2951', 16.0), ..., ('1004', 0.0),
       ('100', 0.0), ('1', 0.0)], 
      dtype=[('id', 'S25'), ('score', '<f8')])